## Importando bibliotecas e bases

In [1]:
# importando bibliotecas
import pandas as pd
import numpy as np
import math
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

## Extração entradas do prompt

In [2]:
def extract_segments_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Separar o conteúdo por "Favor dar somente a resposta a partir do @"
    segments = content.split("Análise de sentimentos.")[1:]  # O primeiro item será ignorado, pois será vazio ou não relevante
    # Análise de sentimentos.

    # Remover qualquer espaço em branco no início ou no fim dos segmentos
    segments = [segment.strip() for segment in segments]

    return segments

In [3]:
from google.colab import drive
drive.mount('/content/drive/')
file_path = '/content/drive/MyDrive/USP/10º semestre/TCC/Prompts/prompts_reddit_k=1_nn.txt'
segments = extract_segments_from_file(file_path)

Mounted at /content/drive/


In [4]:
segments

['Input:  dont hate aap need give them chance \nOutput: @negative\n\nInput:  all aspiring music producer who mainly does hip hop and pop records currently completing higher nitec cert business and want pursue diploma music either nafa laselle but not have any music background and only play and listen ear have never attended music theory lessons though kinda get what and have only played euphonium primary school would like ask would you reccommend try out and what advice would you give when going for the auditions and possible any yall are were the schools what actually learn and how much reccommend kid with background knowledge graded experience join this diploma cheers \nOutput: @',
 'Input: well well this seems fake aspx \nOutput: @negative\n\nInput: and media and congress are responsible for this \nOutput: @',
 'Input: all throughout this period tension there have been few twitter accounts that have foreign names and appear going with what looks like running commentary the troop mov

# Instalação vLLM


In [5]:
# !pip install vllm
!pip install vllm==0.2.1.post1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.6/28.6 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [6]:
from vllm import LLM, SamplingParams

# Inicialização do modelo
llm = LLM(model="facebook/opt-iml-max-1.3b")


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

INFO 12-01 03:26:23 llm_engine.py:72] Initializing an LLM engine with config: model='facebook/opt-iml-max-1.3b', tokenizer='facebook/opt-iml-max-1.3b', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)


tokenizer_config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

INFO 12-01 03:27:48 llm_engine.py:207] # GPU blocks: 3450, # CPU blocks: 1365


In [7]:
sampling_params = SamplingParams(temperature=0, top_p=1, max_tokens=100)
def query_LLM(segment):
    # Adicionando a frase específica ao início do segmento
    modified_segment = "Please only give the answer to the last output. Remember, the aswer must be one of these options: 'positive' or 'negative'. You are only allowed a single word response." + segment
    outputs = llm.generate(modified_segment, sampling_params)

    # Coletar todas as respostas geradas para este segmento
    responses = [o.outputs[0].text.strip() for o in outputs]

    return responses

In [8]:
responses = []

for segment in segments:
    response = query_LLM(segment)
    # print(response)
    responses.append(response)


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s]

WARNING 12-01 03:28:19 scheduler.py:146] Input prompt (2733 tokens) is too long and exceeds limit of 2048


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.99it/s]


In [9]:
print(responses)  # ['positive', 'negative', ...]

[['negative'], ['negative'], ['negative'], ['negative'], ['negative'], ['negative'], ['negative'], ['positive'], ['negative'], ['positive'], ['negative'], ['positive'], ['positive'], ['positive'], ['negative'], ['negative'], ['negative'], ['positive'], ['negative'], ['negative'], ['negative'], ['negative'], ['negative'], ['positive'], ['negative'], ['negative'], ['positive'], ['negative'], ['negative'], ['positive'], ['negative'], ['negative'], ['negative'], ['positive'], ['positive'], ['negative'], ['negative'], ['negative'], ['negative'], ['positive'], ['negative'], ['negative'], ['positive'], ['negative'], ['negative'], ['negative'], ['negative'], ['positive'], ['negative'], ['negative'], ['negative'], ['negative'], ['negative'], ['positive'], ['negative'], ['negative'], ['positive'], ['negative'], ['negative'], ['negative'], ['positive'], ['positive'], ['negative'], ['negative'], ['negative'], ['positive'], ['negative'], ['negative'], ['negative'], ['negative'], ['positive'], ['neg

In [10]:
cleaned_responses = []
cont = 0

for response_group in responses:
    first_response = response_group[0]  # Pegar o primeiro item do grupo de respostas
    first_word = first_response.split('\n')[0].strip()  # Dividir pela quebra de linha e pegar a primeira palavra
    # Tratamento da alucinação do modelo
    if first_word != 'positive' and first_word != 'negative' and first_word != 'neutral':
      first_word = 'negative'
      cont = cont + 1
    cleaned_responses.append(first_word)

print(cleaned_responses)  # ['negative', 'negative', 'neutral', ...]

['negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'po

In [11]:
len(cleaned_responses)

500

# Resultados

In [12]:
import csv
file_csv_llm = '/content/drive/MyDrive/USP/10º semestre/TCC/Resultados/IMDB/opt-iml-max-1.3b/cleaned_responses_reddit_k=1_nn.csv'
file_csv_test = '/content/drive/MyDrive/USP/10º semestre/TCC/Resultados/Dell/opt-iml-max-1.3b/test_responses_reddit_nn.csv'
# Criando um arquivo CSV
with open(file_csv_llm, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Response'])  # Cabeçalho da coluna
    for response in cleaned_responses:
        writer.writerow([response])

# test_df['Labels'].to_csv(file_csv_test, index=False)

In [13]:
import pandas as pd
drive.mount('/content/drive/')
# Ler os dois arquivos CSV
cleaned_responses_df = pd.read_csv(file_csv_llm)
test_df = pd.read_csv(file_csv_test)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [14]:
from sklearn.metrics import classification_report
print(classification_report(test_df, cleaned_responses_df))

              precision    recall  f1-score   support

    negative       0.54      0.94      0.69       243
    positive       0.82      0.25      0.38       257

    accuracy                           0.59       500
   macro avg       0.68      0.60      0.54       500
weighted avg       0.69      0.59      0.53       500

